# COVID-19 Disease Model - Kansas
This notebook will be an attempted implementation of a COVID-19 disease spread model based on the SEIR model paradigm. All of the data I will use is publicly available. 

**Disclosure:** I am employed as an actuary by Blue Cross and Blue Shield of Kansas, Inc. I am also a member of both the Society of Actuaries and the American Academy of Actuaries. Nothing about this model is associated in any way with my employment and none of its conclusions should be taken as financial or actuarial advice. It is merely an exercise for me to learn about these types of models, hopefully alongside others with a shared interest.

All of the data I will use is publicly available. I received no compensation or financial consideration from anyone to create it. My only aim is to learn, and to discover whether a single person, working in his basement, can do a passable job of modeling the apocalypse.

Others have done this better than I will. Among them are:

* [You Yang Gu](https://covid19-projections.com/) - Note that this one doesn't use differential equations. I am going to try it using them
* [IHME](https://covid19.healthdata.org/united-states-of-america)
* [Reich Lab](https://reichlab.io/) - Catalogue of others.

# A. Imports

In [2]:
import pandas as pd
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

# B. Data
* The [COVID Tracking Project](https://covidtracking.com/) - case counts, death counts, hospitalization counts, ICU counts.
* The [Kansas Department of Health and Environment](https://www.coronavirus.kdheks.gov/) - current data and other information; the source for probabilities by age.

In [4]:
df = pd.read_csv("https://covidtracking.com/api/v1/states/ks/daily.csv")

In [7]:
df.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200709,KS,17618,190864,NaN,NaN,1269.0,NaN,397.0,NaN,...,208482,0,0,4da78b703a9dc6b6db59b151e7661e1cd918850a,0,0,0,0,0,NaN
1,20200708,KS,17618,190864,NaN,NaN,1269.0,NaN,397.0,NaN,...,208482,2,34,244672cf48b190600ad7ba7e244725b0375954d4,0,0,0,0,0,NaN
2,20200707,KS,16901,185318,NaN,NaN,1235.0,NaN,384.0,NaN,...,202219,0,0,b6b0ebf1047f1a647de87560327a62f72f22d669,0,0,0,0,0,NaN
3,20200706,KS,16901,185318,NaN,NaN,1235.0,NaN,384.0,NaN,...,202219,3,16,15b09d61855db58f1f6e730eb3db9457660a8347,0,0,0,0,0,NaN
4,20200705,KS,15919,175642,NaN,NaN,1219.0,NaN,377.0,NaN,...,191561,0,0,57a9e89e3e93788666340d0d1c3dffee08d0bde3,0,0,0,0,0,NaN


# C. Parameters and assumptions

## C1. Assumptions

In [ ]:
age_ranges = ['0-9', '10-17', '18-24','25-34','35-44','45-54','55-64','65-74','75-84','85+']
hosp_probs = []